Before we start, we need to make sure that we have a Kafka cluster running and a topic that produces some streaming data. For simplicity, we will use a single-node Kafka cluster and a topic named `users`. Open the `4.0 user-gen-kafka.ipynb` notebook and execute the cell. This notebook produces a user record every few seconds and put it on a Kafka topic called users. 

In [11]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, avg
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [12]:
builder = (SparkSession.builder
           .appName("transform-filter-streaming")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder,['org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1']).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [13]:
df = (spark.readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", "kafka:9092")
      .option("subscribe", "users")
      .option("startingOffsets", "earliest")
      .load())

In [14]:
schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('age', IntegerType(), True),
    StructField('gender', StringType(), True),
    StructField('country', StringType(), True)])

df = df.withColumn('value', from_json(col('value').cast("STRING"), schema))

In [15]:
df = df.select(
    col('value.id').alias('id'),
    col('value.name').alias('name'),
    col('value.age').alias('age'),
    col('value.gender').alias('gender'),
    col('value.country').alias('country'))

In [16]:
df = (df.select('age','country', 'gender').filter("age >= 21").groupBy('country', 'gender').agg(avg('age').alias('avg_age')))

In [17]:
query = (df.writeStream
    .outputMode('complete')
    .format('console')
    .start())

-------------------------------------------
Batch: 0
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|              31.5|
|   Brazil|     M|              34.0|
|      USA|     F|              59.5|
|Australia|     F|              43.0|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|             44.25|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|              31.5|
|   Brazil|     M|              34.0|
|      USA|     F|              59.5|
|Australia|     F|              43.0|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|              45.0|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|              31.5|
|   Brazil|     M|              34.0|
|      USA|     F|              59.5|
|Australia|     F|              40.0|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|              45.0|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|              31.5|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              40.0|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|              45.0|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|              31.5|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              36.6|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|              45.0|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 5
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|              31.5|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              36.6|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|45.666666666666664|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 6
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|38.666666666666664|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              36.6|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|45.666666666666664|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 7
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|38.666666666666664|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              36.6|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|             50.75|
|    India|     M|45.666666666666664|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 8
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|38.666666666666664|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              36.6|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|             50.75|
|    India|     M|45.666666666666664|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              28.5|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 9
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|38.666666666666664|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              36.6|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|             50.75|
|    India|     M|45.666666666666664|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              28.5|
|    India|     F|50.666666666666664|
|       UK|     F|              32.6|
+---------+------+------------------+



In [18]:
query.stop()

24/02/04 17:54:32 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 10, writer: ConsoleWriter[numRows=20, truncate=true]] is aborting.
24/02/04 17:54:32 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 10, writer: ConsoleWriter[numRows=20, truncate=true]] aborted.


In [20]:
spark.stop() 